In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import pprint
import fake_useragent
from fake_useragent import UserAgent
import arxiv
import pandas as pd
import urllib
import feedparser
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from pymongo import MongoClient


Slow version of gensim.models.doc2vec is being used


In [2]:
url = 'https://arxiv.org/archive'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

ua = UserAgent()

user_agent = {'User-agent': ua.random}
print(user_agent)
response  = requests.get(url, headers = user_agent)
print(response)

pprint.pprint(response.text)

{'User-agent': 'Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/30.0.1599.17 Safari/537.36'}
<Response [200]>
('<?xml version="1.0" encoding="UTF-8"?>\n'
 '<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" '
 '"http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n'
 '<html xmlns="http://www.w3.org/1999/xhtml" lang="en">\n'
 '<head>\n'
 '<title>arXiv.org - Archives</title>\n'
 '<link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" />\n'
 '<link rel="stylesheet" type="text/css" media="screen" '
 'href="/css/arXiv.css?v=20170424" />\n'
 '\n'
 '<!-- Piwik -->\n'
 '<script type="text/javascript">\n'
 'var _paq = _paq || [];\n'
 '_paq.push(["setDomains", ["*.arxiv.org"]]);\n'
 "_paq.push(['trackPageView']);\n"
 "_paq.push(['enableLinkTracking']);\n"
 '(function()\n'
 '{ var u="//webanalytics.library.cornell.edu/"; _paq.push([\'setTrackerUrl\', '
 "u+'piwik.php']); _paq.push(['setSiteId', 538]); var d=document, "
 "g=d.createEle

In [3]:
soup=BeautifulSoup(response.content, "lxml")

** Create linksList which will contain all the useful links from the archive page **

In [4]:
linksList=[]
for link in soup.find_all('a'):
    linksList.append(link.get('href'))

In [5]:
archivesList=[]
for link in linksList:
    if 'archive' in str(link):
        archivesList.append(link)
        
print(archivesList)

['/archive/list', '/archive/astro-ph', '/archive/cond-mat', '/archive/cs', '/archive/gr-qc', '/archive/hep-ex', '/archive/hep-lat', '/archive/hep-ph', '/archive/hep-th', '/archive/math', '/archive/math-ph', '/archive/nlin', '/archive/nucl-ex', '/archive/nucl-th', '/archive/physics', '/archive/q-bio', '/archive/q-fin', '/archive/quant-ph', '/archive/stat', '/archive/physics.acc-ph', '/archive/nlin.AO', '/archive/math.AG', '/archive/physics.ao-ph', '/archive/physics.atom-ph', '/archive/physics.data-an', '/archive/nlin.CD', '/archive/physics.chem-ph', '/archive/cs.CL', '/archive/nlin.CG', '/archive/math.DG', '/archive/math.FA', '/archive/cond-mat.mtrl-sci', '/archive/nlin.PS', '/archive/physics.plasm-ph', '/archive/math.QA', '/archive/nlin.SI', '/archive/cond-mat.supr-con']


**Creating function that uses each 'link' in the archivesList and scrapes the page of that link - saving the 'keywords' into a 'otherList' **

In [6]:
masterList=[]
otherList=[]

baseURL='http://www.arxiv.org'
for i in range(1, len(archivesList)):
    print(baseURL+archivesList[i])

http://www.arxiv.org/archive/astro-ph
http://www.arxiv.org/archive/cond-mat
http://www.arxiv.org/archive/cs
http://www.arxiv.org/archive/gr-qc
http://www.arxiv.org/archive/hep-ex
http://www.arxiv.org/archive/hep-lat
http://www.arxiv.org/archive/hep-ph
http://www.arxiv.org/archive/hep-th
http://www.arxiv.org/archive/math
http://www.arxiv.org/archive/math-ph
http://www.arxiv.org/archive/nlin
http://www.arxiv.org/archive/nucl-ex
http://www.arxiv.org/archive/nucl-th
http://www.arxiv.org/archive/physics
http://www.arxiv.org/archive/q-bio
http://www.arxiv.org/archive/q-fin
http://www.arxiv.org/archive/quant-ph
http://www.arxiv.org/archive/stat
http://www.arxiv.org/archive/physics.acc-ph
http://www.arxiv.org/archive/nlin.AO
http://www.arxiv.org/archive/math.AG
http://www.arxiv.org/archive/physics.ao-ph
http://www.arxiv.org/archive/physics.atom-ph
http://www.arxiv.org/archive/physics.data-an
http://www.arxiv.org/archive/nlin.CD
http://www.arxiv.org/archive/physics.chem-ph
http://www.arxiv.org/

In [7]:
tempList=[]
cleanedList=[]

def pullKeywordInfo(archives):
    for link in archives:
        keywords=[]
        baseURL='http://www.arxiv.org'
        currentUrl=baseURL+link
        user_agent = {'User-agent': 'Mozilla/5.0'}
        response=requests.get(currentUrl, headers=user_agent)
        ua = UserAgent()
        user_agent = {'User-agent': ua.random}
        soup=BeautifulSoup(response.content, "lxml")
        generalData=soup.find_all("li")
        testList=soup.find_all("b")
        del testList[:3]
        tempList.extend(testList)

pullKeywordInfo(archivesList)

In [8]:
for i in tempList:
    cleanedList.append(i.text)

** Split off all unnecessary information from string elements - only saving the keyword used in API query (keywords saved into masterList **

In [9]:
for i in range(0, len(cleanedList)):
    masterList.append(cleanedList[i].split(' ')[0])

In [10]:
pprint.pprint(len(masterList))

482


482 keyword items but some are duplicates... turn it into a set, then list:

In [11]:
masterList=list(set(masterList))

In [12]:
len(masterList)

139

## API Query Example #1
    Below is an example of an API query. The entire cell below can be used within a loop, making a query for every item in the masterList. Not necessarily possible because of the arXiv's rate limit. I did the queries in batches so I wouldn't hit the rate limit and the program wouldn't crash

In [ ]:
# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# 'base_url' needs to be complemented with the items below + 'query'
search_query = 'all:' + 'astro-ph.GA'
start = 0                     # retreive the first 5 results
max_results = 10 # max number of results 

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

# Opensearch metadata such as totalResults, startIndex, 
# and itemsPerPage live in the opensearch namespase.
# Some entry metadata lives in the arXiv namespace.
# This is a hack to expose both of these namespaces in
# feedparser v4.1
feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url+query).read()

# change author -> contributors (because contributors is a list)
response = response.replace(b'author',b'contributor')

# parse the response using feedparser
feed = feedparser.parse(response)
print("FEED IS OF THIS TYPE {}".format(type(feed)))
# print out feed information
print( 'Feed title: %s' % feed.feed.title)
print( 'Feed last updated: %s' % feed.feed.updated)

# print opensearch metadata
print( 'totalResults for this query: %s' % feed.feed.opensearch_totalresults)
print( 'itemsPerPage for this query: %s' % feed.feed.opensearch_itemsperpage)
print( 'startIndex for this query: %s'   % feed.feed.opensearch_startindex)

# Run through each entry, and print out information
for entry in feed.entries:
    print( 'e-print metadata')
    print( 'arxiv-id: %s' % entry.id.split('/abs/')[-1])
    print('Published: %s' % entry.published)
    print( 'Title:  %s' % entry.title)
    
    print( 'Authors:  %s' % ','.join(author.name for author in entry.contributors))
    
    # get the links to the abs page and pdf for this e-print
    for link in entry.links:
        if link.rel == 'alternate':
            print( 'abs page link: %s' % link.href)
        elif link.title == 'pdf':
            print('pdf link: %s' % link.href)
    
    # The journal reference, comments and primary_category sections live under 
    # the arxiv namespace
    try:
        journal_ref = entry.arxiv_journal_ref
    except AttributeError:
        journal_ref = 'No journal ref found'
    print( 'Journal reference: %s' % journal_ref)
    
    try:
        comment = entry.arxiv_comment
    except AttributeError:
        comment = 'No comment found'
    print( 'Comments: %s' % comment)
    
    
    print( 'Primary Category: %s' % entry.tags[0]['term'])
    
    # Lets get all the categories
    all_categories = [t['term'] for t in entry.tags]
    print( 'All Categories: %s' % (', ').join(all_categories))
    # The abstract is in the <summary> element
    print( 'Abstract: %s' %  entry.summary)

**Since the API rate limit was too small, I just did a query for each keyword in the masterList and got the 
max entries for each keyword. Adding these values to a 'resultsList' and then eventually zipping the two lists**

In [13]:
resultsList=[]

for i in range(0, len(masterList)):
    # Base api query url
    base_url = 'http://export.arxiv.org/api/query?';

    # Search parameters
    search_query = 'all:' + masterList[i] # search for electron in all fields
    start = 0                     # retreive the first 5 results
    max_results = 1

    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                         start,
                                                         max_results)

    # Opensearch metadata such as totalResults, startIndex, 
    # and itemsPerPage live in the opensearch namespase.
    # Some entry metadata lives in the arXiv namespace.
    # This is a hack to expose both of these namespaces in
    # feedparser v4.1
    feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
    feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # change author -> contributors (because contributors is a list)
    response = response.replace(b'author',b'contributor')

    # parse the response using feedparser
    feed = feedparser.parse(response)
    resultsList.append(int(feed.feed.opensearch_totalresults))

** Zip the two lists: **

In [14]:
zipped=list(zip(masterList,resultsList))
pprint.pprint(zipped)

[('nlin.SI', 8474),
 ('math.GM', 2166),
 ('cs.IR', 3549),
 ('cs.DB', 3191),
 ('physics.data-an', 5798),
 ('physics.atom-ph', 12300),
 ('q-fin.RM', 978),
 ('cs.MS', 989),
 ('cs.SI', 7096),
 ('cs.SD', 842),
 ('math.CA', 12543),
 ('math.OC', 14362),
 ('cs.PL', 2935),
 ('q-fin.EC', 442),
 ('math.IT', 22157),
 ('q-bio.BM', 3125),
 ('math.KT', 3668),
 ('cond-mat.quant-gas', 11025),
 ('physics.gen-ph', 8061),
 ('cond-mat.str-el', 45644),
 ('q-bio.QM', 4225),
 ('cs.NI', 8871),
 ('physics.hist-ph', 2609),
 ('nlin.CG', 1036),
 ('q-bio.PE', 5693),
 ('astro-ph.EP', 11831),
 ('q-fin.CP', 945),
 ('math.NT', 21501),
 ('math.AP', 27834),
 ('q-fin.GN', 1575),
 ('cs.PF', 1280),
 ('math.NA', 12440),
 ('nlin.PS', 6008),
 ('q-bio.GN', 1798),
 ('cs.CC', 5426),
 ('cs.SE', 4093),
 ('math.GN', 2712),
 ('math.OA', 7995),
 ('stat.CO', 2862),
 ('cond-mat.other', 11981),
 ('cs.MA', 1549),
 ('cond-mat.dis-nn', 15578),
 ('q-bio.MN', 2332),
 ('physics.comp-ph', 8486),
 ('physics.plasm-ph', 7946),
 ('physics.ins-det',

sort the zipped list in ascending order of max values:

In [15]:
zipped.sort(key = lambda t: t[1])

In [16]:
zipped[0:10]

[('cs.GL', 130),
 ('physics.app-ph', 250),
 ('cs.OS', 338),
 ('stat.OT', 406),
 ('q-fin.EC', 442),
 ('q-fin.MF', 639),
 ('q-bio.OT', 691),
 ('q-fin.TR', 807),
 ('q-fin.PM', 814),
 ('cs.SD', 842)]

connect to MongoDB:

In [17]:
from pymongo import MongoClient
client = MongoClient()
db=client.test

In [18]:
archives = client.arxivDB.archives

## API Query #2
    I did the queries in keyword batches of 10, until I got to a keyword that would be too much to handle. At that point, I did each single keyword query in multiple batches (if that makes sense).
    The code below continuously adds to my Mongo database.

In [ ]:
for i in range(41,50):
    # Base api query url
    base_url = 'http://export.arxiv.org/api/query?';

    # Search parameters
    search_query = 'all:' + zipped[i][0] # search for electron in all fields
    start = 0                     # retreive the first 5 results
    max_results = zipped[i][1]

    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

    feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
    feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # change author -> contributors (because contributors is a list)
    response = response.replace(b'author',b'contributor')
    
    # parse the response using feedparser
    feed = feedparser.parse(response)
    for i in range(0, len(feed.entries)):
        db.archives.insert_one(feed.entries[i])

In [ ]:
for i in range(95,96):
    # Base api query url
    base_url = 'http://export.arxiv.org/api/query?';

    # Search parameters
    search_query = 'all:' + zipped[i][0] # search for electron in all fields
    start = 0                     # retreive the first 5 results
    max_results = zipped[i][1]

    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

    feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
    feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # change author -> contributors (because contributors is a list)
    response = response.replace(b'author',b'contributor')

    # parse the response using feedparser
    feed = feedparser.parse(response)
    #print(type(feed))
    #print(type(feed.entries))
    #archives.insert_one(feed.entries)
    for i in range(0, len(feed.entries)):
        db.archives.insert_one(feed.entries[i])

## maxed out at item 95... need to create function that makes api queries in batches

In [ ]:
def createBatches(item):
    maxRes=item[1]
    batches = maxRes // 500
    remainder = maxRes-(batches*500)
    print(item[1])
    print(batches)
    print(remainder)
    return batches, remainder


def apiQuery(item, batches, remainder):
    counter=1
    while counter<= batches:
        base_url = 'http://export.arxiv.org/api/query?';

        # Search parameters
        search_query = 'all:' + item[0] # search for electron in all fields
        start = (counter-1)*500                     # retreive the first 5 results
        max_results = (counter*500)

        query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                         start,
                                                         max_results)

        feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
        feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

        # perform a GET request using the base_url and query
        response = urllib.request.urlopen(base_url+query).read()

        # change author -> contributors (because contributors is a list)
        response = response.replace(b'author',b'contributor')

        # parse the response using feedparser
        feed = feedparser.parse(response)
        #print(type(feed))
        #print(type(feed.entries))
        #archives.insert_one(feed.entries)
        for i in range(0, len(feed.entries)):
            db.archives.insert_one(feed.entries[i])
        counter+=1
    
    base_url = 'http://export.arxiv.org/api/query?';

    # Search parameters
    search_query = 'all:' + item[0] # search for electron in all fields
    start = batches*500                     # retreive the first 5 results
    max_results = remainder

    query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

    feedparser._FeedParserMixin.namespaces['http://a9.com/-/spec/opensearch/1.1/'] = 'opensearch'
    feedparser._FeedParserMixin.namespaces['http://arxiv.org/schemas/atom'] = 'arxiv'

    # perform a GET request using the base_url and query
    response = urllib.request.urlopen(base_url+query).read()

    # change author -> contributors (because contributors is a list)
    response = response.replace(b'author',b'contributor')

    # parse the response using feedparser
    feed = feedparser.parse(response)
    #print(type(feed))
    #print(type(feed.entries))
    #archives.insert_one(feed.entries)
    for i in range(0, len(feed.entries)):
        db.archives.insert_one(feed.entries[i])

Check to see how many documents are in my MongoDB:

In [ ]:
db.archives.count()

In [ ]:
df=pd.DataFrame(list(db.archives.find()))

** Save arxivDF to csv: **

In [ ]:
#df.to_csv('arxivDF_v2.csv')

** Read csv into DataFrame: **

In [ ]:
#arxivDF=pd.read_csv('arxivDF_v2.csv', sep=',')